In [143]:
from PIL import Image
import numpy as np
import random

maze_image = Image.open('5x5.png')


In [144]:
# Converts image to a defaut tuple of RBG .convert() or greyscale .convert("L") or a Palletescale .convert("P")
# Greyscale averages the RBG tuple into a single number so its easier to work with
maze_image = maze_image.convert('P')


In [145]:
# resize image to fit scale of matrix
# Helps to convert some random scale of the image into a scale we can use
maze_image = maze_image.resize((5, 5))


In [146]:
# converts image to array

image_array = np.array(maze_image)

print(image_array)


[[1 1 0 0 2]
 [1 0 0 1 0]
 [0 0 0 1 0]
 [0 1 0 1 0]
 [3 1 0 0 0]]


In [147]:
#image_array = np.pad(image_array, pad_width=1, mode='constant', constant_values=1)
# print(image_array)


In [148]:
states = 25
actions = 4
state_action_table = np.zeros((states, actions))
print(state_action_table)


[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [169]:
def create_rewards_matrix(maze):
    rewards = np.full_like(maze, -1, dtype=int)
    for y, row in enumerate(maze):
        for x, cell in enumerate(row):
            if cell == 3:  # Charging station
                rewards[y][x] = 50
            elif cell == 1:  # Wall
                rewards[y][x] = -10
            elif cell == 2:  # Charging station
                rewards[y][x] = 0
    return rewards


x = create_rewards_matrix(image_array)
print(x)


[[-10 -10  -1  -1   0]
 [-10  -1  -1 -10  -1]
 [ -1  -1  -1 -10  -1]
 [ -1 -10  -1 -10  -1]
 [ 50 -10  -1  -1  -1]]


In [174]:
start_state = np.where(image_array == 2)
row_index = start_state[0][0]
column_index = start_state[1][0]
start_state = (row_index, column_index)
print(start_state)

# Set the parameters for the Q-learning algorithm
num_episodes = 100
alpha = 0.8
gamma = 0.95
epsilon = 0.1


(0, 4)


In [168]:
num_states, num_actions = state_action_table.shape
i, j = start_state
state = i * num_states + j

action = np.random.choice(state_action_table[state])
i, j = divmod(action, num_states)
print(i, j)
#next_state = i * num_states + j
#reward = state_action_table[i, j]
# print(state_action_table)


0.0 0.9999872


In [170]:

# Define the hyperparameters
alpha = 0.1  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 0.1  # Exploration rate

# Define the function to convert state coordinates to an index in the Q-table
def state_to_index(state):
    return state[0], state[1]

# Define the function to select an action based on the Q-table and the exploration rate
def select_action(state, q_table, epsilon):
    if np.random.random() < epsilon:
        # Select a random action
        action = np.random.choice([0, 1, 2, 3])
    else:
        # Select the action with the highest Q-value
        index = state_to_index(state)
        action = np.argmax(q_table[index])
    return action

# Define the main training loop
for episode in range(10000):
    # Reset the environment to the starting state
    state = (1, 1)
    done = False
    
    while not done:
        # Select an action based on the Q-table and the exploration rate
        action = select_action(state, q_table, epsilon)
        
        # Take the selected action and observe the next state and reward
        if action == 0:  # Move up
            next_state = (state[0]-1, state[1])
        elif action == 1:  # Move down
            next_state = (state[0]+1, state[1])
        elif action == 2:  # Move left
            next_state = (state[0], state[1]-1)
        elif action == 3:  # Move right
            next_state = (state[0], state[1]+1)
        
        # Check if the next state is valid (i.e., within the bounds of the environment)
        if (next_state[0] >= 0 and next_state[0] < 5 and next_state[1] >= 0 and next_state[1] < 5):
            # Get the reward for the next state
            reward = env[next_state]
            
            # Update the Q-value for the current state and action
            index = state_to_index(state)
            q_table[index][action] += alpha * (reward + gamma * np.max(q_table[state_to_index(next_state)]) - q_table[index][action])
            
            # Move to the next state
            state = next_state
        else:
            # If the next state is invalid, penalize the action and stay in the current state
            index


SyntaxError: 'break' outside loop (1131792370.py, line 12)

In [171]:
updatedQtable = Qlearning(image_array, state_action_table,
                          start_state, num_episodes, alpha, gamma, epsilon)
print(updatedQtable)


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices